## Welcome to tutorial02 about [AQW](https://arxiv.org/abs/2306.00978) quantization

### A. How to download a LLM: the website of [Huggingface](https://huggingface.co/)


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [5]:
list(model.state_dict())

['model.embed_tokens.weight',
 'model.layers.0.self_attn.q_proj.weight',
 'model.layers.0.self_attn.q_proj.bias',
 'model.layers.0.self_attn.k_proj.weight',
 'model.layers.0.self_attn.k_proj.bias',
 'model.layers.0.self_attn.v_proj.weight',
 'model.layers.0.self_attn.v_proj.bias',
 'model.layers.0.self_attn.o_proj.weight',
 'model.layers.0.mlp.gate_proj.weight',
 'model.layers.0.mlp.up_proj.weight',
 'model.layers.0.mlp.down_proj.weight',
 'model.layers.0.input_layernorm.weight',
 'model.layers.0.post_attention_layernorm.weight',
 'model.layers.1.self_attn.q_proj.weight',
 'model.layers.1.self_attn.q_proj.bias',
 'model.layers.1.self_attn.k_proj.weight',
 'model.layers.1.self_attn.k_proj.bias',
 'model.layers.1.self_attn.v_proj.weight',
 'model.layers.1.self_attn.v_proj.bias',
 'model.layers.1.self_attn.o_proj.weight',
 'model.layers.1.mlp.gate_proj.weight',
 'model.layers.1.mlp.up_proj.weight',
 'model.layers.1.mlp.down_proj.weight',
 'model.layers.1.input_layernorm.weight',
 'model.l

### B. the website of official AWQ github : [llm-awq](https://github.com/mit-han-lab/llm-awq)

**How to install the virtual environment for "llm-awq":**

**1. clone the repository: (make sure that you have installed "[git](https://git-scm.com/downloads)"):**
```bash
cd 【the path you want to save the repository】
git clone https://github.com/mit-han-lab/llm-awq.git
```

**2. create a virtual environment:**
```bash
conda create -n awq python=3.10 -y
conda activate awq
```

**3. install the packge locally:**
```bash
cd llm-awq
pip install -e .
cd awq/kernels
python setup.py install
pip install flash-attn --no-build-isolation
```
the installation file of ['flash-attn'](https://github.com/Dao-AILab/flash-attention/releases/)

you can use "**pip list**" | "**python --ersion**" | "**nvidia-smi**" to see the version of all packages installed:

![](./figs/torch_version.png)
![](./figs/python_version.png)
![](./figs/cuda_version.png)

![](./figs/flash_version.png)


```bash
pip install 【the path the file saved】
```

**4. How to use AWQ:**
**(1) Perform AWQ search and save search results (we already did it for you):**
```bash
python -m awq.entry --model_path /home/ET/jtfan/MyData/Models/DeepSeek-R1-Distill-Qwen-1.5B \
    --w_bit 4 --q_group_size 128 \
    --run_awq --dump_awq awq_cache/DeepSeek-R1-Distill-Qwen-1.5B-w4-g128.pt
```
![](./figs/awq-step01.png)


**(2) Generate real quantized weights (INT4):**
```bash
mkdir quant_cache
python -m awq.entry --model_path /home/ET/jtfan/MyData/Models/DeepSeek-R1-Distill-Qwen-1.5B \
    --w_bit 4 --q_group_size 128 \
    --load_awq awq_cache/DeepSeek-R1-Distill-Qwen-1.5B-w4-g128.pt \
    --q_backend real --dump_quant quant_cache/DeepSeek-R1-Distill-Qwen-1.5B-w4-g128-awq.pt
```
![](./figs/awq-step02.png)


**(3) Load and evaluate the real quantized model (now you can see smaller gpu memory usage):**
```bash
python -m awq.entry --model_path /home/ET/jtfan/MyData/Models/DeepSeek-R1-Distill-Qwen-1.5B \
    --tasks wikitext \
    --w_bit 4 --q_group_size 128 \
    --load_quant quant_cache/DeepSeek-R1-Distill-Qwen-1.5B-w4-g128-awq-v2.pt
```
![](./figs/awq-step03.png)

### B. LLM inference test:

**1. Download a LLM in [Huggingface](https://huggingface.co/)**
Almost every large language model will have a awq-quantized version in huggingface after using better and larger dataset for AWQ finetuing, so these models have better performance.
![](./figs/awq-models.png)

**2. Install AutoAWQ**
```bash
conda create -n autoawq python=3.10 -y
conda activate autoawq
pip install torch==2.3.0
pip install tansformers==4.56.2
cd ./AutoAWQ-0.2.7
pip install -e .
pip istall triton==3.1.0
```

In [6]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer

device = 'cuda:3'
model_path = '/home/ET/jtfan/MyData/Models/DeepSeek-R1-Distill-Qwen-1.5B'
download_quant_path = '/home/ET/jtfan/MyData/Models/casperhansen/deepseek-r1-distill-qwen-1.5b-awq'
quantized_model = AutoAWQForCausalLM.from_quantized(download_quant_path)
tokenizer = AutoTokenizer.from_pretrained(download_quant_path, trust_remote_code=True)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Convert prompt to tokens
prompt_template = """\
<|system|>
</s>
<|user|>
{prompt}</s>
<|assistant|>"""

prompt = "You're standing on the surface of the Earth. " \
         "You walk one mile south, one mile west and one mile north. " \
         "You end up exactly where you started. Where are you?"

tokens = tokenizer(
    prompt_template.format(prompt=prompt),
    return_tensors='pt'
).input_ids.to(device)

# Generate output
generation_output = quantized_model.generate(
    tokens,
    streamer=streamer,
    max_length=1024,
)


Replacing layers...: 100%|██████████| 28/28 [00:04<00:00,  5.91it/s]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 
Okay, so I'm trying to figure out this problem where someone walks one mile south, then one mile west, and then one mile north, and ends up where they started. Hmm, interesting. I remember hearing about this kind of problem before, maybe in a math or logic class. Let me think through it step by step.
First, the person starts on the Earth's surface. They walk one mile south. So, they're moving south from some point. Then, they walk one mile west. Now, if they're near the poles, walking west could bring them back to a point where they're aligned north-south with their starting point. Then, they walk one mile north back to where they started. That makes sense. So, one possible answer is that they're near the South Pole.
But wait, are there other possibilities? Like, maybe they're not near the South Pole. Maybe they're somewhere else where walking west brings them back to a point that's one mile north of their starting point. Let me think about that.
So, if they start near the South Pole

In [7]:
list(quantized_model.state_dict())

['model.model.embed_tokens.weight',
 'model.model.layers.0.self_attn.q_proj.qweight',
 'model.model.layers.0.self_attn.q_proj.qzeros',
 'model.model.layers.0.self_attn.q_proj.scales',
 'model.model.layers.0.self_attn.q_proj.bias',
 'model.model.layers.0.self_attn.k_proj.qweight',
 'model.model.layers.0.self_attn.k_proj.qzeros',
 'model.model.layers.0.self_attn.k_proj.scales',
 'model.model.layers.0.self_attn.k_proj.bias',
 'model.model.layers.0.self_attn.v_proj.qweight',
 'model.model.layers.0.self_attn.v_proj.qzeros',
 'model.model.layers.0.self_attn.v_proj.scales',
 'model.model.layers.0.self_attn.v_proj.bias',
 'model.model.layers.0.self_attn.o_proj.qweight',
 'model.model.layers.0.self_attn.o_proj.qzeros',
 'model.model.layers.0.self_attn.o_proj.scales',
 'model.model.layers.0.mlp.gate_proj.qweight',
 'model.model.layers.0.mlp.gate_proj.qzeros',
 'model.model.layers.0.mlp.gate_proj.scales',
 'model.model.layers.0.mlp.up_proj.qweight',
 'model.model.layers.0.mlp.up_proj.qzeros',
 'm

### C. How to evaluate a LLM and AWQ-Quantized-LLM:

**1. install evaluation package "[lm-aluation-harness](https://github.com/EleutherAI/lm-evaluation-harness)"**

![](./figs/harness.png)


**Paper [Sheared LLaMA: Accelerating Language Model Pre-training via Structured Pruning](https://arxiv.org/abs/2310.06694)**
![](./figs/sheared-llama.png)



```bash
cd /home/ET/jtfan/github_download/lm-evaluation-harness
pip install -e .
```

```bash
lm_eval --model hf --model_args pretrained=/home/ET/jtfan/MyData/Models/DeepSeek-R1-Distill-Qwen-1.5B --tasks sciq --device cuda:2 --batch_size auto:4 --num_fewshot 4
```


![](./figs/sciq-eval.png)

**Using "evaluation_more.py" :**

![](./figs/evaluation_more.png)

**Question : lm-evaluation-harness not support AWQ-Qantized-LLM:**

![](./figs/eval_bug.png)

![](./figs/fig1.png)

**One of the solutions :**
![](./figs/fig2.png)

```bash
cd AutoAWQ-0.2.7\awq\modules\linear\gemm.py
```

![](./figs/gemm_kernel.png)

![](./figs/results.png)